# User-Based Collaborative Filtering Analysis

This notebook demonstrates how to evaluate a user-based collaborative filtering system using leave-one-out cross-validation. The system uses cosine similarity to find similar users and makes recommendations based on their ratings, helping us understand how well the algorithm performs for different types of users.

## Setup and Imports

First, let's import the necessary libraries and load our data.

In [1]:
from recsys.MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
from recsys.RecommenderMetrics import RecommenderMetrics
from recsys.EvaluationDataset import EvaluationDataset

ml, data, rankings = MovieLens.load()

## Prepare Evaluation Data

Set up the evaluation data using leave-one-out cross-validation (LOOCV) to test our recommender system. This approach helps us understand how well the system performs for individual users.

In [2]:
evalData = EvaluationDataset(data, rankings)

# Train on leave-One-Out train set
train_set = evalData.GetLOOCVTrainSet()
leftOutTestSet = evalData.GetLOOCVTestSet()

## Train the Model

Train a KNN-based model using cosine similarity for user-based collaborative filtering. This will help us identify similar users and their preferences.

In [3]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}

model = KNNBasic(sim_options=sim_options)
model.fit(train_set)
simsMatrix = model.compute_similarities()

## Generate Recommendations

For each user, we'll:
1. Find their k most similar users
2. Get movies rated by similar users
3. Weight the ratings by user similarity
4. Generate top-N recommendations

This process helps us understand how well the system can predict user preferences based on similar users' behavior.

In [4]:
# Build up dict to lists of (int(movieID), predictedrating) pairs
topN = defaultdict(list)
k = 10  # Number of similar users to consider

for uiid in range(train_set.n_users):
    # Get top N similar users to this one
    similarityRow = simsMatrix[uiid]
    
    similarUsers = []
    for innerID, score in enumerate(similarityRow):
        if (innerID != uiid):
            similarUsers.append((innerID, score))
    
    kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])
    
    # Get the stuff they rated, and add up ratings for each item, weighted by user similarity
    candidates = defaultdict(float)
    for similarUser in kNeighbors:
        innerID = similarUser[0]
        userSimilarityScore = similarUser[1]
        theirRatings = train_set.ur[innerID]
        for rating in theirRatings:
            candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
    
    # Build a dictionary of stuff the user has already seen
    watched = {}
    for itemID, rating in train_set.ur[uiid]:
        watched[itemID] = 1
    
    # Get top-rated items from similar users:
    pos = 0
    for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            movieID = train_set.to_raw_iid(itemID)
            topN[int(train_set.to_raw_uid(uiid))].append((int(movieID), 0.0))
            pos += 1
            if (pos > 40):
                break

## Evaluate Performance

Measure the hit rate of our recommendations to see how well the system performs. This metric tells us how often our recommendations match the user's actual preferences.

In [5]:
# Measure hit rate
print("HR", RecommenderMetrics.HitRate(topN, leftOutTestSet))